# Enhancing RAG with Neo4j Knowledge Graph


Import modules and instantiate connections and models

In [2]:
import fundus

import config
from graph import NewsGraphClient
from ner import EntityFinder

/home/djordje/.virtualenvs/buzz24/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Crawl news sites
publishers = (fundus.PublisherCollection.de, fundus.PublisherCollection.uk)
crawler = fundus.Crawler(*publishers)
articles = list(crawler.crawl(max_articles=2))
for article in articles:
    print(article)

Fundus-Article:
- Title: "Diese Abgeordneten aus Mitteldeutschland ziehen ins Europaparlament ein"
- Text:  "Kein Land stellt mehr Abgeordnete im Europaparlament als Deutschland. Insgesamt
          96 Sitze entfallen auf deutsche Politiker. Zehn von ihnen sind [...]"
- URL:    https://www.mdr.de/nachrichten/deutschland/europaparlament-abgeordnete-mitteldeutschland-100.html
- From:   Mitteldeutscher Rundfunk (MDR) (2024-06-10 17:20)
Fundus-Article:
- Title: "Niederländer de Jong verpasst die EM"
- Text:  "Einer ihrer bekanntesten Spieler wird den Niederlanden bei der
          Europameisterschaft fehlen. Barcelona-Profi de Jong wird nach [...]"
- URL:    https://www.kicker.de/niederlaender-de-jong-verpasst-die-em-1031137/artikel#omrss
- From:   Kicker (2024-06-10 23:47)


## Perform some queries
e.g. list a few sources

In [6]:
db = NewsGraphClient()
records = db.query("MATCH (s:Source) RETURN s.name,  as source LIMIT $limit", limit=10)
for record in records:
    print(record)

{'source': 'Junge Welt'}
{'source': 'Stern'}
{'source': 'Hamburger Abendblatt'}
{'source': 'Westdeutscher Rundfunk'}
{'source': 'Mitteldeutscher Rundfunk (MDR)'}
{'source': 'Süddeutsche Zeitung'}
{'source': 'Deutsche Welle'}
{'source': 'Focus Online'}
{'source': 'zdfHeute'}
{'source': 'Münchner Merkur'}


## Perform Named Entity Recognition
with GLiNER

In [4]:
entity_finder = EntityFinder(config.RELEVANT_LABELS)

for article in articles:   
    entities_in_headline = entity_finder.find(article.title)
    print(article.title, 'mentions the following entities:', *(f"{e.label}: {e.name} |" for e in entities_in_headline))
    entities_in_text = entity_finder.find(article.plaintext)
    print('The text also mentions the following entities:', *(f"{e.label}: {e.name} |" for e in entities_in_text))

/home/djordje/.virtualenvs/buzz24/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/home/djordje/.virtualenvs/buzz24/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Diese Abgeordneten aus Mitteldeutschland ziehen ins Europaparlament ein mentions the following entities: person: Abgeordneten| location: Mitteldeutschland| organization: Europaparlament|


/home/djordje/.virtualenvs/buzz24/lib/python3.10/site-packages/gliner/modules/data.py:93: UserWarning: Sentence of length 522 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


The text also mentions the following entities: organization: Europaparlament| location: Mitteldeutschland| location: Straßburg| location: Brüssel| organization: CDU| person: Oliver Schenk| organization: sächsische AfD| person: Siegbert Droese| organization: Europaparlament| person: Björn Höcke| person: Maximilian Krah| location: Brüssel| organization: Europaparlament| person: Carola Rackete| organization: sächsische Linke| location: Schleswig-Holstein| person: Anna Cavazzini| location: Brüssel| organization: Die Grünen| organization: Europaparlament| person: Matthias Ecke| organization: Europaparlament| person: Constanze Krehl| location: Sachsen-Anhalt| location: Sachsen-Anhalt| organization: Europaparlament| organization: CDU Sachsen-Anhalt| person: Alexandra Mehnert| location: Brüssel|
Niederländer de Jong verpasst die EM mentions the following entities: person: Niederländer de Jong|
The text also mentions the following entities: location: Niederlanden| location: Kanada| location: Is